Bokeh Hover作成

In [ ]:
from bokeh.plotting import figure, show, output_notebook, save, output_file
from bokeh.sampledata.iris import flowers
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, Legend, HoverTool, LinearAxis, Range1d    # 2軸用
output_notebook()

In [ ]:
def create_figure(datatype, p0=''):
    p = figure(title=datatype, plot_width=plot_w, plot_height=plot_h, tools=TOOLS)
    p.yaxis.axis_label = 'ratio'
    if p0 != '':
        p = figure(title=datatype, plot_width=plot_w, plot_height=plot_h, x_range=p0.x_range, tools=TOOLS)
    if datatype == '18dop':
        p.xaxis.axis_label = 'Number of days since Oct. 1 2020.'
    
    for k, v in first_val.items():
    #     p.line(df_type.index, df_type[k].values / first_val[k], color=color_dic[k], legend=k)
        src = ColumnDataSource(data={'x_arr':df_type.index, 'y_arr':df_type[k].values / first_val[k], 'vn':[k]*len(df_type)})
        p.line(x='x_arr', y='y_arr', source=src, color=color_dic[k], legend=k)
    p.line(df_type.index, 1, color='black', line_dash=(4, 4), line_width=0.5)
    p.add_tools(HoverTool(tooltips=[('vehiclename','@vn'),('days','@x_arr'),('ratio','@y_arr')]))
    return p

In [ ]:
TOOLS = "pan,wheel_zoom,reset,save"
plot_w, plot_h = 850, 300

datatype = '15cyrc'
df_type, first_val, color_dic = select_dtype(datatype)
p = create_figure(datatype, '')

datatype = '17cy'
df_type, first_val, color_dic = select_dtype(datatype)
p1 = create_figure(datatype, p)

output_file('/home/maeyamada/data/bokeh.html', title='car count')
show(gridplot([[p],[p1],[p2],[p3]]))

In [ ]:
TOOLS = "pan,wheel_zoom,reset,save"
plot_w, plot_h = 900, 300

def create_figure2(datatype, p0=''):
    df_type = df_vn_count[df_vn_count['dtype']==datatype].dropna(axis=1, how='all').fillna(0).reset_index(drop=True)
    data_dic = dict(df_type.iloc[:,4:].astype(np.int32))
    data_dic['day_arr'] = df_type.index
    data_dic['day_str'] = [f'{y:04}-{m:02}-{d:02}' for y,m,d in zip(df_type['yyyy'], df_type['mm'], df_type['dd'])]    # hover用
    src = ColumnDataSource(data=data_dic)
    names = src.column_names[:-2]    # day_arr と day_str以外
    color_scale = LinearColormap(['deepskyblue','lime','yellow','tomato'], vmin=0, vmax=len(names))
    color_arr = [color_scale(i) for i in range(len(names))]
    
    p = figure(title=datatype, plot_width=plot_w, plot_height=plot_h, tools=TOOLS)
    if p0 != '':
        p = figure(title=datatype, plot_width=plot_w, plot_height=plot_h, x_range=p0.x_range, tools=TOOLS)
    if datatype == '18dop':
        p.xaxis.axis_label = 'Number of days since Oct. 1 2020.'

    renderers = p.vbar_stack(stackers=names, x='day_arr', width=0.9, color=color_arr, source=src, name=names)
    # hover 作成
    # p.add_tools(HoverTool(tooltips=[('vehiclename','$name'),('days','@day_arr')]))
    for r in renderers:
        vn = r.name
        hover = HoverTool(tooltips=[("vehiclename", f"{vn}"), ("value", f"@{vn}"), ('day', '@day_str ($index days)')], renderers=[r])
        p.add_tools(hover)
    legend = Legend(items=[(x, [renderers[i]]) for i, x in enumerate(names)][::-1], location=(0, 18*(10-len(names))))
    if datatype == '17plus':
        legend = Legend(items=[(x, [renderers[i]]) for i, x in enumerate(names)][::-1], location=(0, -40),
                        spacing=-9, glyph_height=12, label_text_font_size='6pt')
    legend.click_policy = 'hide'    # 凡例クリックの非表示対応 (少し期待外れだけど)
    p.add_layout(legend, 'right')

    return p